# Demo the tool that checks for the presence of bright stars in the Claws susceptibility region

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pysiaf.utils import rotations
import matplotlib.patches as patches
import pysiaf
from matplotlib.path import Path
import os

import warnings
warnings.filterwarnings('ignore')

from jwst_rogue_path_tool.src.utils import get_config
from jwst_rogue_path_tool.src.claws_tool import apt_program, compute_line
from jwst_rogue_path_tool.src.apt_sql import Sqlfile


SETTINGS = get_config()
DATA_PATH = SETTINGS['jwst_rogue_path_data']

%matplotlib notebook

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-062 CANNOT BE CHECKED AGAINST ONLINE VERSION


## Common catalog defintions

In [2]:
catargsSIMBAD={'inner_rad':8.,'outer_rad':12.,'sourcecat':'SIMBAD',
                                       'band':'K','maxmag':4.,'simbad_timeout':200,'verbose':True}
catargs2MASS={'inner_rad':8.,'outer_rad':12.,'sourcecat':'2MASS','2MASS_filename':os.path.join(DATA_PATH, 'TwoMassForMario_km5.csv'),
                                       'band':'k_m','maxmag':4,'verbose':True}

## First use case: review a program for potential presence of claws at any orientation

### Create a program object (ingests and parses an APT sql file)

In [3]:
filename = os.path.join(DATA_PATH, 'APT_data', 'Demo_apt.sql')
program = apt_program(filename)

### Define properties of the catalog query

### Select which observations to test (use None to test all the observations)

In [4]:
obsids = [2,3]

#### First ingest the observations and get their corresponding catalog

In [5]:
program.add_observations(obsids=obsids, catargs=catargs2MASS, smallregion=False)

Adding obsid: 2
Total number of NIRCAM exposures in this observation: 6
Total number of NIRCAM exposure specifications: 3
Total number of visits: 3
Adopted inner and outer radius 7.963386910783606 12.036613089216395
Adding obsid: 3
Total number of NIRCAM exposures in this observation: 4
Total number of NIRCAM exposure specifications: 2
Total number of visits: 2
Adopted inner and outer radius 7.963473984763377 12.036526015236623


#### Run the checks on the possible V3PA values

In [6]:
program.check_observations()

Checking obsid: 2
Checking obsid: 3


#### Plot the results

In [7]:
for ob in program.observations:
    ob.plot_observations_checks()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

*** Valid ranges ****
PA Start -- PA End: 3.5 -- 73.5
PA Start -- PA End: 76.5 -- 78.5
PA Start -- PA End: 111.5 -- 113.5
PA Start -- PA End: 296.0 -- 298.0
PA Start -- PA End: 326.0 -- 336.5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

*** Valid ranges ****
PA Start -- PA End: 4.0 -- 73.0
PA Start -- PA End: 77.5 -- 78.5
PA Start -- PA End: 112.0 -- 113.5
PA Start -- PA End: 295.5 -- 298.5
PA Start -- PA End: 326.0 -- 337.0


## Second use case: verify that an assigned PA works well for a yet unexecuted observation

### Create a program object (ingests and parses an APT sql file)

In [8]:
savedirroot = SETTINGS['output_directory']
PID = '4446'
filename = os.path.join(DATA_PATH, 'APT_data', 'PID{}.sql'.format(PID))
program = apt_program(filename)

savedir = savedirroot+'PID_'+PID+'/'

if os.path.isdir(savedir) ==  False:
    os.mkdir(savedir)



### Select which observation(s) to check

In [9]:
obsids = [3]

### Ingest and check them

In [10]:
program.add_observations(obsids=obsids,catargs=catargs2MASS,smallregion=False)
program.check_observations()

Adding obsid: 3
Total number of NIRCAM exposures in this observation: 12
Total number of NIRCAM exposure specifications: 3
Total number of visits: 1
Adopted inner and outer radius 7.997095168432756 12.002904831567244
Checking obsid: 3


### Planned V3PA

In [11]:
att = np.linspace(159,181,12)
print(att)

[159. 161. 163. 165. 167. 169. 171. 173. 175. 177. 179. 181.]


In [14]:
rds = []
for a in att:

    rd = program.observations[0].fixed_angle(a, RP_padding=0.0, nrows=1, 
                                             ncols=2,draw_allexp=False, 
                                             savefilenames=[None, savedir + 'PID_{}_{:6.2f}.png'.format(PID, a)])
    rds.append(rd)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
#for rd in rds:
#    print(rd['V3PA'],rd['totcts_B'])
#    display(rd['catalog'].loc[rd['BM_ALL_B'],['RAdeg','DECdeg','k_m','avg_intensity_B','j_m_fict_mag_B','h_m_fict_mag_B','k_m_fict_mag_B']])

In [16]:
program.observations

## Third use case: check an executed program at its fixed V3PA 

Can be used to see:

    - whether we have stars in the susceptibility region for observations that showed claws
    - whether we DO NOT have stars in the susceptibility region for observations that DO NOT show claws



### Subcase A :rely on the observation having been reported as affected by claws (ie the obs is listed in Ben's "claw_summary.txt" table)

#### Read in the claws_summary file

This file contains a list of observations that showed claws, and their executed V3PA

##### Pick one of the cases

In [37]:
PID, obsid = 2738, 3
BM = (df_obs_claws['PID'] == PID) & (df_obs_claws['obs'] == obsid)
df_obs_claws[BM]

NameError: name 'df_obs_claws' is not defined

#### Read in the corresponding SQL file

In [23]:
filename = os.path.join(DATA_PATH, 'APT_data','PID{}.sql'.format(PID))

#### Create a program object

In [24]:
program = apt_program(filename)

#### Add just the affected observation

In [25]:
program.add_observations(obsids=[obsid],catargs=catargs2MASS,
                         smallregion=False)

NameError: name 'obsid' is not defined

#### Check the observation

In [27]:
program.check_observations()

#### Run some summary diagnostic and get plots and a dataframe with the results

In [28]:
rd = program.observations[0].fixed_angle(df_obs_claws.loc[BM,'pa_v3'].values[0],RP_padding=0.,nrows=2,ncols=3)

IndexError: list index out of range

In [29]:
rd['catalog'].head()

,RAdeg,DECdeg,j_m,h_m,k_m,avg_intensity_A,avg_intensity_B,avg_V2,avg_V3,j_m_fict_mag_A,j_m_fict_mag_B,h_m_fict_mag_A,h_m_fict_mag_B,k_m_fict_mag_A,k_m_fict_mag_B
319,169.619775,33.094265,1.073,0.183,0.282,0.0,0.0,1.687186,9.259887,99.0,99.0,99.0,99.0,99.0,99.0
847,184.952945,48.984138,2.080,1.277,1.042,0.0,0.0,-8.537434,-7.452489,99.0,99.0,99.0,99.0,99.0,99.0
1412,164.886632,36.093090,2.464,1.605,1.471,0.0,0.0,5.505288,6.121400,99.0,99.0,99.0,99.0,99.0,99.0
1516,163.327953,34.214947,2.047,1.426,1.530,0.0,0.0,6.925372,7.900350,99.0,99.0,99.0,99.0,99.0,99.0
1747,184.031484,40.660210,2.605,1.709,1.644,0.0,0.0,-9.269591,0.864881,99.0,99.0,99.0,99.0,99.0,99.0


In [30]:
df_A = rd['catalog'].filter(regex=("IN_A_exp*"))

BM_ALL_A = True
for c in df_A.columns:
    BM_ALL_A &= df_A[c]

df_B = rd['catalog'].filter(regex=("IN_B_exp*"))

BM_ALL_B = True
for c in df_B.columns:
    BM_ALL_B &= df_B[c]



print('Ben table')
display(df_obs_claws.loc[BM,['detector','claws','notes']])
print(' ')

print('Mod A results')
display(rd['catalog'].loc[BM_ALL_A,['RAdeg','DECdeg','k_m']])
print(' ')

print('Mod B results')
display(rd['catalog'].loc[BM_ALL_B,['RAdeg','DECdeg','k_m']])
print(' ')


### Subcase B : more general, does not rely on the observation being listed in Ben's "claw_summary.txt" table

#### Setup PID and obs and v3PA

In [31]:
PID, obsid, pa_v3 = 1181, 98, 133.3177706698362

#### Read in the corresponding SQL file, setup proram and observation

In [32]:
filename = os.path.join(DATA_PATH, 'APT_data', 'PID_{}.sql'.format(PID))
program = apt_program(filename)
program.add_observations(obsids=[obsid],catargs=catargs2MASS,smallregion=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mfix/Desktop/data/APT_data/PID_1181.sql'

#### Check and run diagnostic for fixed angle

In [33]:
program.check_observations()
rd = program.observations[0].fixed_angle(pa_v3,RP_padding=0.,nrows=2,ncols=3)

IndexError: list index out of range

## Run a detailed sweep in V3 PA on one obs to determne the DN/pix/ks in claws measured on the detector and compare them to the expected background

### Setup PID and obs and v3PA

In [34]:
#PID, obsid, pa_v3 = 1181, 98, 133.3177706698362
PID, obsid, pa_v3 = 1187, 35, 94.13

### Read in the corresponding SQL file, setup proram and observation

In [35]:
filename = '/Users/gennaro/Desktop/Claws_prediction_tool/APT_sql/PID_{}.sql'.format(PID)
program = apt_program(filename)
program.add_observations(obsids=[obsid],catargs=catargs2MASS,smallregion=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gennaro/Desktop/Claws_prediction_tool/APT_sql/PID_1187.sql'

### Run the sweep

In [36]:
tmS,tcS = program.observations[0].DN_report(np.arange(0.,360.,3),RP_padding=0.,draw_reports=True,
                                                      background_params=[{'threshold':0.1,'func':np.min},{'threshold':0.2,'func':np.mean}])
#,
#                                                     save_report_dir='/Users/gennaro/Desktop/testc/',save_figures_dir='/Users/gennaro/Desktop/testc/')
#tmA,tmB, tcA, tcB = program.observations[0].DN_report(np.arange(0.,360.,1),RP_padding=0.,draw_reports=True,savefilename=None,
#                                                      background_params=[{'threshold':0.1,'func':np.min}])

IndexError: list index out of range